In [7]:
import pandas as pd
import json

In [8]:
file_path = './Clinical_and_Other_Features.xlsx'
data = pd.read_excel(file_path, sheet_name='Data')

In [6]:
print("Columns in the dataset:")
print(data.columns)

Columns in the dataset:
Index(['Patient Information', 'MRI Technical Information', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Demographics', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Tumor Characteristics', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'MRI Findings',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'SURGERY',
       'Unnamed: 54', 'Unnamed: 55', 'Radiation Thera

In [11]:
race_and_ethnicity_col = data.columns[20]
# Print the Race and Ethnicity column to verify
try:
    print(data[race_and_ethnicity_col])
except KeyError as e:
    print(f"KeyError: {e}. Please check if the column names match exactly.")

Unnamed: 20
